### Original Kernel with minimal variations

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.set_printoptions(threshold=200)
pd.set_option('display.max_columns', 200)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data/"))

# Any results you write to the current directory are saved as output.

['device_data.csv', 'pageviews_complemento.csv', 'sampleSubmission.csv', 'SITE_ID.csv', 'PAGE.csv', 'README.md', 'CONTENT_CATEGORY.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'pageviews.csv', 'conversiones.csv', 'CONTENT_CATEGORY_TOP.csv']


In [27]:
data = pd.concat([
       pd.read_csv("../data/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("../data/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])

/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data.shape

In [ ]:
data.head()

In [28]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [ ]:
X_test.head(5)

In [ ]:
X_test.shape

In [ ]:
X_test.iloc[0,0:65].sum() # la suma de las frecuencias debe ser 1

In [29]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [ ]:
X_train.sample(5)

In [ ]:
X_train.iloc[0,0:65].sum()

In [30]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [31]:
y_prev = pd.read_csv("../data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(set(X_train.index))
y_train.loc[list(idx)] = 1

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=250)
fit = test.fit(X_train, y_train)
X_train = pd.DataFrame(fit.transform(X_train))
X_test = pd.DataFrame(fit.transform(X_test))

In [ ]:
print(X_train.shape, X_test.shape)

In [32]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    learner = LGBMClassifier(n_estimators=10000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)])
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("respuesta", header=True, compression="zip")
fi = pd.concat(fi, axis=1).mean(axis=1)

[1]	training's binary_logloss: 0.128942	training's auc: 0.834728	valid_1's binary_logloss: 0.152446	valid_1's auc: 0.8069
Training until validation scores don't improve for 10 rounds.
[2]	training's binary_logloss: 0.117939	training's auc: 0.877138	valid_1's binary_logloss: 0.14837	valid_1's auc: 0.806152
[3]	training's binary_logloss: 0.11014	training's auc: 0.889498	valid_1's binary_logloss: 0.144616	valid_1's auc: 0.813151
[4]	training's binary_logloss: 0.10407	training's auc: 0.908865	valid_1's binary_logloss: 0.144056	valid_1's auc: 0.80524
[5]	training's binary_logloss: 0.0987176	training's auc: 0.918161	valid_1's binary_logloss: 0.141111	valid_1's auc: 0.81437
[6]	training's binary_logloss: 0.0940959	training's auc: 0.92979	valid_1's binary_logloss: 0.140652	valid_1's auc: 0.816399
[7]	training's binary_logloss: 0.090013	training's auc: 0.947593	valid_1's binary_logloss: 0.139541	valid_1's auc: 0.81687
[8]	training's binary_logloss: 0.0863618	training's auc: 0.952338	valid_1's b

[15]	training's binary_logloss: 0.0665778	training's auc: 0.979306	valid_1's binary_logloss: 0.138332	valid_1's auc: 0.844475
[16]	training's binary_logloss: 0.0643723	training's auc: 0.983213	valid_1's binary_logloss: 0.138313	valid_1's auc: 0.844053
[17]	training's binary_logloss: 0.0622859	training's auc: 0.985654	valid_1's binary_logloss: 0.137106	valid_1's auc: 0.847463
[18]	training's binary_logloss: 0.0604267	training's auc: 0.98678	valid_1's binary_logloss: 0.136286	valid_1's auc: 0.85007
[19]	training's binary_logloss: 0.0584142	training's auc: 0.989747	valid_1's binary_logloss: 0.136189	valid_1's auc: 0.849007
[20]	training's binary_logloss: 0.0567267	training's auc: 0.991952	valid_1's binary_logloss: 0.13647	valid_1's auc: 0.847262
[21]	training's binary_logloss: 0.054903	training's auc: 0.994711	valid_1's binary_logloss: 0.136503	valid_1's auc: 0.84658
[22]	training's binary_logloss: 0.0532951	training's auc: 0.995738	valid_1's binary_logloss: 0.136139	valid_1's auc: 0.8476

[8]	training's binary_logloss: 0.0857571	training's auc: 0.951319	valid_1's binary_logloss: 0.136791	valid_1's auc: 0.851203
[9]	training's binary_logloss: 0.0822581	training's auc: 0.9609	valid_1's binary_logloss: 0.136129	valid_1's auc: 0.848937
[10]	training's binary_logloss: 0.0792266	training's auc: 0.965225	valid_1's binary_logloss: 0.135127	valid_1's auc: 0.848977
[11]	training's binary_logloss: 0.0762722	training's auc: 0.968027	valid_1's binary_logloss: 0.134871	valid_1's auc: 0.84652
[12]	training's binary_logloss: 0.0737495	training's auc: 0.970009	valid_1's binary_logloss: 0.134309	valid_1's auc: 0.846741
[13]	training's binary_logloss: 0.0712652	training's auc: 0.971952	valid_1's binary_logloss: 0.133286	valid_1's auc: 0.847343
[14]	training's binary_logloss: 0.0688965	training's auc: 0.973233	valid_1's binary_logloss: 0.132501	valid_1's auc: 0.848887
[15]	training's binary_logloss: 0.0668561	training's auc: 0.975527	valid_1's binary_logloss: 0.132621	valid_1's auc: 0.8458

[18]	training's binary_logloss: 0.0618373	training's auc: 0.98614	valid_1's binary_logloss: 0.104359	valid_1's auc: 0.798649
[19]	training's binary_logloss: 0.0598298	training's auc: 0.989987	valid_1's binary_logloss: 0.103992	valid_1's auc: 0.810597
[20]	training's binary_logloss: 0.0582219	training's auc: 0.991157	valid_1's binary_logloss: 0.10386	valid_1's auc: 0.811858
[21]	training's binary_logloss: 0.0564414	training's auc: 0.992485	valid_1's binary_logloss: 0.103617	valid_1's auc: 0.811799
[22]	training's binary_logloss: 0.0548896	training's auc: 0.993757	valid_1's binary_logloss: 0.103908	valid_1's auc: 0.808281
[23]	training's binary_logloss: 0.0532894	training's auc: 0.994739	valid_1's binary_logloss: 0.103593	valid_1's auc: 0.811294
[24]	training's binary_logloss: 0.0517988	training's auc: 0.995608	valid_1's binary_logloss: 0.104217	valid_1's auc: 0.808445
[25]	training's binary_logloss: 0.0504922	training's auc: 0.99662	valid_1's binary_logloss: 0.104107	valid_1's auc: 0.80

### Experimental models

En este caso conv contiene los resultados para enviar al concurso, pero a continuacion subdividiremos X_train para poder usar el periodo de octubre a diciembre como testeo.

In [3]:
data = data[data.FEC_EVENT.dt.month < 10]
X_test2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test2 = pd.concat(X_test2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [5]:
data = data[(data.FEC_EVENT.dt.month < 10) & (data.FEC_EVENT.dt.month > 6)]
X_train2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train2 = pd.concat(X_train2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [4]:
y_prev2 = pd.read_csv("../data/conversiones.csv")
y_test2 = pd.Series(0, index=X_test2.index)
idx = set(y_prev2[y_prev2.mes >= 10].USER_ID.unique()).intersection(set(X_test2.index))
y_test2.loc[list(idx)] = 1

In [6]:
y_train2 = pd.Series(0, index=X_train2.index)
idx = set(y_prev2[(y_prev2.mes < 10) & (y_prev2.mes > 6)].USER_ID.unique()).intersection(set(X_train2.index))
y_train2.loc[list(idx)] = 1

In [7]:
print(X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape)

(11356, 1434) (11356,) (11529, 2026) (11529,)


In [8]:
features = list(set(X_train2.columns).intersection(set(X_test2.columns)))
X_train2 = X_train2[features]
X_test2 = X_test2[features]

In [10]:
X_train2.shape

(11356, 1434)

In [11]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train2, y_train2)
parciales = dtc.predict(X_test2)

In [12]:
from sklearn.metrics import roc_auc_score

scores = roc_auc_score(y_test2,parciales)

In [13]:
scores

0.6589190854421179

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=200)
fit = test.fit(X_train2, y_train2)
X_train2 = pd.DataFrame(fit.transform(X_train2))
X_test2 = pd.DataFrame(fit.transform(X_test2))

In [18]:
print(X_train2.shape, X_test2.shape)

(11356, 200) (11529, 200)


In [61]:
from sklearn.tree import DecisionTreeClassifier

dtc2 = DecisionTreeClassifier()
dtc2.fit(X_train2, y_train2)
parciales = dtc2.predict(X_test2)
scores = roc_auc_score(y_test2,parciales)

scores

0.6635218739759915

In [66]:
prob.shape

(11529, 2)